# Food Deserts in DC

I like this concept! If executed well, this can be a really great project.

Explain that food deserts have many definitions, but for urban areas, let us consider it as a place that has low access to a healthy food retail outlet or supermarket within 1 mile of a residence. 

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Google API Key
from config import gkey

## Application Variables

Describe briefly what this section does and how the variables work.

In [ ]:
# Origin address or location
origin_location = "1700 2nd St NW Washington, DC"

# Search radius ( 1 mile = ~1600 meters)
origin_radius = 1600 

# Google API endpoints
geocode_url = 'https://maps.googleapis.com/maps/api/geocode/json'
nearbysearch_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Empty list to store grocery stores
grocery_store_list = []

## Data Processing Function

To work around the Harris Teeter, Giant, and Safeway issue, we made two calls to the API. One for the keyword `grocery`, and the other to find type `grocery_or_supermarket`. Look at the code and you should be able to interpret it all.

In [ ]:
def populateGroceryStoreList(response):
    
    base_url = 'https://maps.googleapis.com/maps/api/distancematrix/json'

    for r in response['results']:
        try:
            place_id = r['place_id']
            lat = r['geometry']['location']['lat']
            lng = r['geometry']['location']['lng']
            destination_latlng = f'{lat},{lng}'
            business_status = r['business_status']          
            name = r['name']
            vicinity = r['vicinity']
            try:
                price_level = r['price_level']
            except:
                price_level = None
            try:
                rating = r['rating']
            except:
                rating = None

            params = {
                'units': 'imperial',
                'origins': origin_latlng,
                'destinations': destination_latlng,
                'key': gkey
            }

            distance_response = requests.get(base_url, params=params)
            # print(distance_response.url)
            distance_response = distance_response.json()

            try:
                distance_text = distance_response['rows'][0]['elements'][0]['distance']['text']
                distance_value = distance_response['rows'][0]['elements'][0]['distance']['value']
                duration_text = distance_response['rows'][0]['elements'][0]['duration']['text']
                duration_value = distance_response['rows'][0]['elements'][0]['duration']['value']

                grocery_store_list.append({
                    'place_id': place_id,
                    'name': name,
                    'business_status': business_status,
                    'price_level': price_level,
                    'rating': rating,
                    'vicinity': vicinity,
                    'distance_text': distance_text,
                    'distance_value': distance_value,
                    'duration_text': duration_text,
                    'duration_value': duration_value,
                })

            except Exception as e:
                print(e)

        except Exception as e:
            print(e)

## Geocode Address

Explain what's happening here

In [ ]:
params = {
    'address': origin_location,
    'key': gkey
}

response = requests.get(geocode_url, params=params)
response = response.json()

lat = response["results"][0]["geometry"]["location"]["lat"]
lng = response["results"][0]["geometry"]["location"]["lng"]

origin_latlng = f"{lat},{lng}"

## `NearbySearch` API Call

This one uses the `grocery` as the keyword. 

In [ ]:
params = {
    'location': origin_latlng,
    'radius': origin_radius,
    'keyword': 'grocery',
    'key': gkey
}

response = requests.get(nearbysearch_url, params=params)
print(response.url)
response = response.json()

# call function to populate grocery stores
populateGroceryStoreList(response)

## `NearbySearch` API Call

This one uses the `grocery_or_supermarket` as the type. 

In [ ]:
params = {
    'location': origin_latlng,
    'radius': origin_radius,
    'type': "grocery_or_supermarket",
    #'type': "grocery_or_supermarket|supermarket|food",
    "key": gkey
}

response = requests.get(nearbysearch_url, params=params)
print(response.url)
response = response.json()

# call function to populate grocery stores
populateGroceryStoreList(response)

## Prepare DataFrame

Creating the DataFrame, dropping duplicates, sorting, and filtering for grocery stores within 1 mile of the origin.

In [ ]:
grocery_store_df = pd.DataFrame(grocery_store_list)
grocery_store_df.drop_duplicates(subset='place_id', inplace=True)
grocery_store_df = grocery_store_df.sort_values(by='distance_value').reset_index(drop=True)
grocery_stores_within_1mi = grocery_store_df.loc[grocery_store_df['distance_value'] <= 1600]

## Display the Result

Display the result. Now that you've got `place_id`, you can use other Google APIs to get more information about that place. This is very solid start. You should consider building on top of this.

In [ ]:
grocery_stores_within_1mi